# Pokemon Battle Winner Predictor

## Datasets
combats.csv (train data) = https://raw.githubusercontent.com/saurabhcharde/Pokemon_Battle_Winner_Prediction_using_ML/master/datasets/combats.csv

pokemon_data.csv (features) = https://raw.githubusercontent.com/saurabhcharde/Pokemon_Battle_Winner_Prediction_using_ML/master/datasets/pokemons_data.csv

tests.csv (testing data) = https://raw.githubusercontent.com/saurabhcharde/Pokemon_Battle_Winner_Prediction_using_ML/master/datasets/tests.csv

## Step 1: import data

In [1]:
import pandas as pd
import numpy as np

In [2]:
pokemon=pd.read_csv('https://raw.githubusercontent.com/saurabhcharde/Pokemon_Battle_Winner_Prediction_using_ML/master/datasets/pokemons_data.csv',index_col=0)
combats=pd.read_csv('https://raw.githubusercontent.com/saurabhcharde/Pokemon_Battle_Winner_Prediction_using_ML/master/datasets/combats.csv')

In [3]:
pokemon.head()

,Name,Hit Points,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary
#,,,,,,,,
1,Bulbasaur,45,49,49,65,65,45,0
2,Ivysaur,60,62,63,80,80,60,0
3,Venusaur,80,82,83,100,100,80,0
4,Mega Venusaur,80,100,123,122,120,80,0
5,Charmander,39,52,43,60,50,65,0


## Step 2: Data preprocessing

In [4]:
cols = ["First_pokemon","Second_pokemon","Winner"]
new_combat_data=combats[cols].replace(pokemon.Name)
new_combat_data.head()

,First_pokemon,Second_pokemon,Winner
0,Larvitar,Nuzleaf,Nuzleaf
1,Virizion,Terrakion,Terrakion
2,Togetic,Beheeyem,Beheeyem
3,Slugma,Druddigon,Druddigon
4,Omastar,Shuckle,Omastar


In [5]:
combats.Winner[combats.Winner == combats.First_pokemon] = 0
combats.Winner[combats.Winner == combats.Second_pokemon] = 1

## Step 3: Normalising data and adding features

In [6]:
def normalization(data_df):
    stats=["Hit Points","Attack","Defense","Sp. Atk","Sp. Def","Speed","Legendary"]
    stats_df=pokemon[stats].T.to_dict("list")
    one=data_df.First_pokemon.map(stats_df)
    print(one, len(one))
    two=data_df.Second_pokemon.map(stats_df)
    print(two, len(two))
    temp_list=[]
    for i in range(len(one)):
        temp_list.append(np.array(one[i])-np.array(two[i]))
    new_test = pd.DataFrame(temp_list, columns=stats)
    for c in stats:
        description=new_test[c].describe()
        print(description)
        new_test[c]=(new_test[c]-description['min'])/(description['max']-description['min'])
    return new_test

In [7]:
data=normalization(combats)
data = pd.concat([data,combats.Winner], axis=1)
data.head()

0             [50, 64, 50, 45, 50, 41, 0]
1           [91, 90, 72, 90, 129, 108, 1]
2            [55, 40, 85, 80, 105, 40, 0]
3             [40, 40, 40, 70, 40, 20, 0]
4           [70, 60, 125, 115, 70, 55, 0]
                       ...               
49995    [100, 120, 100, 150, 120, 90, 1]
49996         [60, 85, 40, 30, 45, 68, 0]
49997        [60, 50, 100, 85, 70, 65, 0]
49998        [40, 30, 50, 55, 55, 100, 0]
49999     [78, 104, 78, 159, 115, 100, 0]
Name: First_pokemon, Length: 50000, dtype: object 50000
0          [70, 70, 40, 60, 40, 60, 0]
1        [91, 129, 90, 72, 90, 108, 1]
2         [75, 75, 75, 125, 95, 40, 0]
3         [77, 120, 90, 60, 90, 48, 0]
4         [20, 10, 230, 10, 230, 5, 0]
                     ...              
49995      [30, 40, 70, 70, 25, 60, 0]
49996      [35, 55, 40, 45, 40, 60, 0]
49997     [73, 115, 60, 60, 60, 90, 0]
49998      [25, 35, 70, 95, 55, 45, 0]
49999      [70, 80, 50, 35, 35, 35, 0]
Name: Second_pokemon, Length: 50000, dtype: object 50

,Hit Points,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary,Winner
0,0.455556,0.468571,0.517241,0.450425,0.527578,0.450746,0.5,1
1,0.500000,0.374286,0.452874,0.543909,0.597122,0.507463,0.5,1
2,0.455556,0.385714,0.517241,0.365439,0.527578,0.507463,0.5,1
3,0.417778,0.257143,0.379310,0.521246,0.383693,0.423881,0.5,1
4,0.611111,0.628571,0.252874,0.790368,0.119904,0.656716,0.5,0


## Step 4: Train-Test Split

In [8]:
x_label=data.drop("Winner",axis=1)
y_label=data["Winner"]

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_label, y_label, test_size=0.25, random_state=42)

## Step 5: Training the model

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
clf = RandomForestClassifier(n_estimators=100)
model = clf.fit(x_train, y_train)
pred = model.predict(x_test)
print('Accuracy of model: ', accuracy_score(pred, y_test)*100)

Accuracy of model:  94.952


## Step 6: Testing the model

In [11]:
test_data=pd.read_csv('https://raw.githubusercontent.com/saurabhcharde/Pokemon_Battle_Winner_Prediction_using_ML/master/datasets/tests.csv')


In [12]:
new_test_data=test_data[["First_pokemon","Second_pokemon"]].replace(pokemon.Name)
new_test_data.head()

,First_pokemon,Second_pokemon
0,Seaking,Lickitung
1,Ferrothorn,Quagsire
2,Thundurus Therian Forme,Hitmonlee
3,Flaaffy,Maractus
4,Spearow,Alomomola


In [13]:
final_data=normalization(test_data)
pred=model.predict(final_data)
test_data["Winner"]=[test_data["First_pokemon"][i] if pred[i]==0 else test_data["Second_pokemon"][i] for i in range(len(pred))]

0          [80, 92, 65, 65, 80, 68, 0]
1        [74, 94, 131, 54, 116, 20, 0]
2       [79, 105, 70, 145, 80, 101, 1]
3          [70, 55, 55, 80, 60, 45, 0]
4          [40, 60, 30, 31, 31, 70, 0]
                     ...              
9995       [60, 60, 60, 85, 85, 85, 0]
9996       [50, 50, 95, 40, 50, 35, 0]
9997       [68, 90, 65, 50, 55, 82, 0]
9998       [75, 87, 63, 87, 63, 98, 0]
9999       [50, 40, 85, 40, 65, 25, 0]
Name: First_pokemon, Length: 10000, dtype: object 10000
0         [90, 55, 75, 60, 75, 30, 0]
1         [95, 85, 85, 65, 65, 35, 0]
2       [50, 120, 53, 35, 110, 87, 0]
3        [75, 86, 67, 106, 67, 60, 0]
4        [165, 75, 80, 40, 45, 65, 0]
                    ...              
9995    [70, 110, 70, 115, 70, 90, 0]
9996    [55, 84, 105, 114, 75, 52, 0]
9997     [35, 45, 160, 30, 45, 70, 0]
9998      [45, 63, 37, 65, 55, 95, 0]
9999     [70, 80, 70, 80, 70, 110, 0]
Name: Second_pokemon, Length: 10000, dtype: object 10000
count    10000.000000
mean        -0.370

In [14]:
combats_name = test_data[cols].replace(pokemon.Name)
combats_name[63:64]

,First_pokemon,Second_pokemon,Winner
63,Mr. Mime,Psyduck,Mr. Mime


## Step 7: Making the predictor

In [15]:
first_pokemon = input("Enter first pokemon: ")
second_pokemon = input("Enter second pokemon: ")
new_test_data = pd.DataFrame({"First_pokemon": [first_pokemon.title()], "Second_pokemon": [second_pokemon.title()]})
new_test_data

Enter first pokemon: pikachu
Enter second pokemon: squirtle


,First_pokemon,Second_pokemon
0,Pikachu,Squirtle


In [16]:
first_dex = pokemon[pokemon['Name'] == first_pokemon.title()].index
second_dex = pokemon[pokemon['Name'] == second_pokemon.title()].index
test_data = pd.DataFrame({"First_pokemon": [first_dex.tolist()[0]], "Second_pokemon": [second_dex.tolist()[0]]})
test_data

,First_pokemon,Second_pokemon
0,31,10


In [17]:
def new_normalization(data_df):
    stats=["Hit Points","Attack","Defense","Sp. Atk","Sp. Def","Speed","Legendary"]
    stats_df=pokemon[stats].T.to_dict("list")
    one=data_df.First_pokemon.map(stats_df)
    print(one, len(one))
    two=data_df.Second_pokemon.map(stats_df)
    print(two, len(two))
    temp_list=[]
    for i in range(len(one)):
        temp_list.append(np.array(one[i])-np.array(two[i]))
        print(temp_list)
    new_test = pd.DataFrame(temp_list, columns=stats)
    return new_test

In [18]:
final_data=new_normalization(test_data)
print(final_data)
pred=model.predict(final_data)
test_data["Winner"]=[test_data["First_pokemon"][i] if pred[i]==0 else test_data["Second_pokemon"][i] for i in range(len(pred))]
combats_name = test_data[cols].replace(pokemon.Name)
combats_name

0    [35, 55, 40, 50, 50, 90, 0]
Name: First_pokemon, dtype: object 1
0    [44, 48, 65, 50, 64, 43, 0]
Name: Second_pokemon, dtype: object 1
[array([ -9,   7, -25,   0, -14,  47,   0])]
   Hit Points  Attack  Defense  Sp. Atk  Sp. Def  Speed  Legendary
0          -9       7      -25        0      -14     47          0


,First_pokemon,Second_pokemon,Winner
0,Pikachu,Squirtle,Pikachu
